# Process SmolLM Corpus

In [1]:
from datasets import load_dataset

cv2 = load_dataset("HuggingFaceTB/smollm-corpus", "cosmopedia-v2")
fed = load_dataset("HuggingFaceTB/smollm-corpus", "fineweb-edu-dedup")
pe = load_dataset("HuggingFaceTB/smollm-corpus", "python-edu")

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/423 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/234 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/1906 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

In [2]:
cv2, fed, pe

(DatasetDict({
     train: Dataset({
         features: ['prompt', 'text', 'token_length', 'audience', 'format', 'seed_data'],
         num_rows: 39134000
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['text', 'id', 'metadata'],
         num_rows: 190168005
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['blob_id', 'repo_name', 'path', 'length_bytes', 'score', 'int_score'],
         num_rows: 7678448
     })
 }))

In [3]:
fed["train"][0]

{'text': "No. 24; Updated March 2011\nClick here to download and print a PDF version of this document.\nParents are usually the first to recognize that their child has a problem with emotions or behavior. Still, the decision to seek professional help can be difficult and painful for a parent. The first step is to gently try to talk to the child. An honest open talk about feelings can often help. Parents may choose to consult with the child's physicians, teachers, members of the clergy, or other adults who know the child well. These steps may resolve the problems for the child and family.\nFollowing are a few signs which may indicate that a child and adolescent psychiatric evaluation will be useful.\n- Marked fall in school performance\n- Poor grades in school despite trying very hard\n- Severe worry or anxiety, as shown by regular refusal to go to school, go to sleep or take part in activities that are normal for the child's age\n- Frequent physical complaints\n- Hyperactivity; fidgeti

In [8]:
fed4 = fed.filter(lambda x: x['metadata']['int_score'] >= 4, num_proc=120)

In [4]:
pe4 = pe.filter(lambda x: x['score'] >= 4.16, num_proc=120)

In [10]:
fed4['train'].num_rows/fed['train'].num_rows,pe4['train'].num_rows/pe['train'].num_rows

(0.1193753228888319, 0.11253237633438425)

In [6]:
from huggingface_hub import login

login()

In [11]:
# pe4.push_to_hub("chengjunyan1/smollm-10","python-edu")
# fed4.push_to_hub("chengjunyan1/smollm-10","fineweb-edu-dedup")

Uploading the dataset shards:   0%|          | 0/229 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/518 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/chengjunyan1/smollm-10/commit/a956dc95743f869a67c3622f0311fe48a695b1e7', commit_message='Upload dataset (part 00004-of-00005)', commit_description='', oid='a956dc95743f869a67c3622f0311fe48a695b1e7', pr_url=None, pr_revision=None, pr_num=None)